In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import scorer, accuracy_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.model_selection import train_test_split
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, make_scorer
from time import time
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, BaggingClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.externals import joblib
import xgboost as xgb
import pickle
import nltk
nltk.download('punkt')
import string
import re
import nltk
from nltk.tokenize import TweetTokenizer

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
!pip install tensorflow
!pip install tensorflow-hub
!pip install sentencepiece
import tensorflow as tf
import tensorflow_hub as hub
print("TF version: ", tf.__version__)
print("Hub version: ", hub.__version__)
import pandas as pd
import numpy as np
import spacy
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import regex as re
import string
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_colwidth', None)
import tensorflow.keras
import sklearn
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, Input, Dropout, GlobalAveragePooling1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, BaggingClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

     |████████████████████████████████| 1.2MB 15.0MB/s 
TF version:  2.4.1
Hub version:  0.11.0


In [8]:
data=pd.read_csv('/content/drive/MyDrive/sexism/final_dataset.csv')
data

,Tweet Id,Tweets,User Id,Screen Name,Class
0,319399851215433000,Oh yeah Colin! Smash those girls! :D #MKR,949380854,CreatrixKali,sexism
1,320817818222358000,It's insane they keep bringing people back. When will this show end #MKR,297877558,quincepoacher,none
2,324114200450437000,"@berkeley_eagle #MKR this shit show has more comebacks than Johnny Farnham, ok back to the #TheVoiceAu",272704749,MarkTramby,none
3,326286656854454000,*sigh* oh Colin 😍 #MKR,71416635,BinnyD,none
4,381988216292655000,"RT @brian_day15: I swear, I'm not sexist, but I honestly just cannot stand the woman college football announcer on ESPN2",930620467,YesYoureSexist,sexism
...,...,...,...,...,...
11320,588296672226058000,"Oh fuck me hard with a rusty chainsaw, another round of instant restaurants??? #MKR",15641984,spanner77,none
11321,590818760337924000,OMG SHUT UP DRASKO AND BIANCA #MKR #FINALFIVE,287061857,amandagavin,none
11322,592273294562209000,STFU drasko #MKR,347048522,SydneyEditor1,none
11323,595166563516329000,Ash found her inner bogun #whistle #fingersinthemouth Ewwww #MKR,27608469,Catatwitering,sexism


In [9]:
data=data.drop(columns=['Tweet Id','User Id','Screen Name'])
data

,Tweets,Class
0,Oh yeah Colin! Smash those girls! :D #MKR,sexism
1,It's insane they keep bringing people back. When will this show end #MKR,none
2,"@berkeley_eagle #MKR this shit show has more comebacks than Johnny Farnham, ok back to the #TheVoiceAu",none
3,*sigh* oh Colin 😍 #MKR,none
4,"RT @brian_day15: I swear, I'm not sexist, but I honestly just cannot stand the woman college football announcer on ESPN2",sexism
...,...,...
11320,"Oh fuck me hard with a rusty chainsaw, another round of instant restaurants??? #MKR",none
11321,OMG SHUT UP DRASKO AND BIANCA #MKR #FINALFIVE,none
11322,STFU drasko #MKR,none
11323,Ash found her inner bogun #whistle #fingersinthemouth Ewwww #MKR,sexism


In [10]:
import numpy as np

In [11]:
data['Class'] = data.Class.map({'none':0, 'sexism':1,'racism':2})

In [12]:
for i in range(len(data)):
  if(data['Class'][i]==2):
    data=data.drop(i)

In [13]:
data

,Tweets,Class
0,Oh yeah Colin! Smash those girls! :D #MKR,1
1,It's insane they keep bringing people back. When will this show end #MKR,0
2,"@berkeley_eagle #MKR this shit show has more comebacks than Johnny Farnham, ok back to the #TheVoiceAu",0
3,*sigh* oh Colin 😍 #MKR,0
4,"RT @brian_day15: I swear, I'm not sexist, but I honestly just cannot stand the woman college football announcer on ESPN2",1
...,...,...
11320,"Oh fuck me hard with a rusty chainsaw, another round of instant restaurants??? #MKR",0
11321,OMG SHUT UP DRASKO AND BIANCA #MKR #FINALFIVE,0
11322,STFU drasko #MKR,0
11323,Ash found her inner bogun #whistle #fingersinthemouth Ewwww #MKR,1


In [14]:
x=data.iloc[0:,0:1]
y=data.iloc[0:,1]

In [15]:
x=x.to_numpy()

In [16]:
from imblearn.over_sampling import RandomOverSampler
os=RandomOverSampler(ratio=1)
x_train,y_train=os.fit_resample(x.reshape(-1,1),y)
df= pd.DataFrame(x_train,columns =['tweet'])
df1=pd.DataFrame(y_train,columns=['label'])
r=pd.concat([df,df1],axis=1)
r

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


,tweet,label
0,Oh yeah Colin! Smash those girls! :D #MKR,1
1,It's insane they keep bringing people back. When will this show end #MKR,0
2,"@berkeley_eagle #MKR this shit show has more comebacks than Johnny Farnham, ok back to the #TheVoiceAu",0
3,*sigh* oh Colin 😍 #MKR,0
4,"RT @brian_day15: I swear, I'm not sexist, but I honestly just cannot stand the woman college football announcer on ESPN2",1
...,...,...
16627,"@avacadosoup @Shahan_Amin Good for you. Too bad about those hurt by it or unprotected by it, right?",1
16628,"RT @lewisinho @OdemwingPlay If you have a vagina, don't try and be funny.",1
16629,"@avacadosoup @Shahan_Amin Good for you. Too bad about those hurt by it or unprotected by it, right?",1
16630,@g0ssipsquirrelx Take a guess...what %?,1


In [17]:
import string
import re
import nltk
from nltk.tokenize import TweetTokenizer

In [34]:
nlp = spacy.load("en")
sp = spacy.load('en_core_web_sm')

nltk.download('stopwords')
nltk.download('punkt')
spacy_st = nlp.Defaults.stop_words
nltk_st = stopwords.words('english')

def clean(tweet, http = True, punc = True, lem = True, stop_w = True):
    
    if http is True:
        tweet = re.sub("https?:\/\/t.co\/[A-Za-z0-9]*", '', tweet)
    if stop_w == 'nltk':
        tweet = [word for word in word_tokenize(tweet) if not word.lower() in nltk_st]
        tweet = ' '.join(tweet)

    elif stop_w == 'spacy':
        tweet = [word for word in word_tokenize(tweet) if not word.lower() in spacy_st]
        tweet = ' '.join(tweet)

    if lem == True:
        lemmatized = [word.lemma_ for word in sp(tweet)]
        tweet = ' '.join(lemmatized)

    if punc is True:
        tweet = tweet.translate(str.maketrans('', '', string.punctuation))

    tweet = re.sub("\s+", ' ', tweet)
    
    return tweet

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [19]:
t=[]
for i in range(len(r)):
  a=str(r['tweet'][i])
  v=re.sub('@[A-Za-z0-9_]*','',a)
  v=re.sub('[0-9]+','',v)
  v=re.sub('RT','',v)
  v=v.lower()
  t.append(v)

In [20]:
g=np.array(t)
d=pd.DataFrame(g,columns=['cleaned_tweet_1'])
r=pd.concat([r,d],axis=1)
r

,tweet,label,cleaned_tweet_1
0,Oh yeah Colin! Smash those girls! :D #MKR,1,oh yeah colin! smash those girls! :d #mkr
1,It's insane they keep bringing people back. When will this show end #MKR,0,it's insane they keep bringing people back. when will this show end #mkr
2,"@berkeley_eagle #MKR this shit show has more comebacks than Johnny Farnham, ok back to the #TheVoiceAu",0,"#mkr this shit show has more comebacks than johnny farnham, ok back to the #thevoiceau"
3,*sigh* oh Colin 😍 #MKR,0,*sigh* oh colin 😍 #mkr
4,"RT @brian_day15: I swear, I'm not sexist, but I honestly just cannot stand the woman college football announcer on ESPN2",1,": i swear, i'm not sexist, but i honestly just cannot stand the woman college football announcer on espn"
...,...,...,...
16627,"@avacadosoup @Shahan_Amin Good for you. Too bad about those hurt by it or unprotected by it, right?",1,"good for you. too bad about those hurt by it or unprotected by it, right?"
16628,"RT @lewisinho @OdemwingPlay If you have a vagina, don't try and be funny.",1,"if you have a vagina, don't try and be funny."
16629,"@avacadosoup @Shahan_Amin Good for you. Too bad about those hurt by it or unprotected by it, right?",1,"good for you. too bad about those hurt by it or unprotected by it, right?"
16630,@g0ssipsquirrelx Take a guess...what %?,1,take a guess...what %?


In [21]:
r=r.drop(['tweet'],axis=1)
r['cleaned_text'] = r.cleaned_tweet_1.apply(lambda x: clean(x, lem = False, stop_w = 'nltk', http = True, punc = True))

In [22]:
r=r.drop(['cleaned_tweet_1'],axis=1)

In [23]:
r

,label,cleaned_text
0,1,oh yeah colin smash girls mkr
1,0,s insane keep bringing people back show end mkr
2,0,mkr shit show comebacks johnny farnham ok back thevoiceau
3,0,sigh oh colin 😍 mkr
4,1,swear m sexist honestly stand woman college football announcer espn
...,...,...
16627,1,good bad hurt unprotected right
16628,1,vagina nt try funny
16629,1,good bad hurt unprotected right
16630,1,take guess


In [24]:
x_train,x_test,y_train,y_test=train_test_split(r['cleaned_text'],r['label'],test_size=0.2)

In [25]:
import pandas as pd
import urllib3
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [26]:
clf = Pipeline([
    ('vect', CountVectorizer(max_features = 10000, ngram_range = (1,2))),
    ('tfidf', TfidfTransformer(norm = 'l2')),
    ('clf', RandomForestClassifier()),
])

In [27]:
clf.fit(x_train,y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=10000, min_df=1,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=No...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                

In [28]:
y_pred=clf.predict(x_test)
accuracy_score(y_pred,y_test)

0.9347760745416291

In [29]:
pickle.dump(clf,open('/content/drive/MyDrive/sexism_2/clf_sex.pickle','wb'))

In [33]:
a=['i am sexist']
clf.predict(a)

array([1])